In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Architecture

In [48]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()  # Python 2.0 & above applicable
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim =  embed_size // heads

        assert(self.head_dim * heads == embed_size)

        # Where are we using these ones in the `forward function`?
        self.queries = nn.Linear(self.embed_size, self.embed_size, bias=False)
        self.keys = nn.Linear(self.embed_size, self.embed_size, bias=False)
        self.values = nn.Linear(self.embed_size, self.embed_size, bias=False)

        self.fc_out = nn.Linear(self.heads * self.head_dim, embed_size)

    def forward(self, query, keys, values, mask: bool):
        N = query.shape[0]  # Training batch size or number of batches?
        query_len, key_len, value_len = query.shape[1], keys.shape[1], values.shape[1]

        queries_proj = self.queries(query)  # (N, Seq, embed_size)
        keys_proj = self.keys(keys)
        values_proj = self.values(values)

        # Split embedding into self.heads pieces
        queries = queries_proj.reshape(N, query_len, self.heads, self.head_dim)  # What is the difference between key_len and N in the first place?
        keys = keys_proj.reshape(N, key_len, self.heads, self.head_dim)
        values = values_proj.reshape(N, value_len, self.heads, self.head_dim)

        energy = torch.einsum("nqhd, nkhd->nhqk", [queries, keys])  # Why an array? & Why this arrangement?
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))
        
        attention = torch.softmax(energy/ self.embed_size ** (1/2), dim=3)

        out = torch.einsum("nhqk,nkhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )  # The shape of the whole must be looked at & how does it reshape this (are the vectors going to the correct place)??

        out = self.fc_out(out)
        return out


In [49]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        """
        The forward expansion term basically takes the vector from a lower dimensional space to a 
        higher dimensional space for better representation capacity. The value used in the paper is
        equal to 4.
        """
        super().__init__()  # Python 3.0 & above applicable
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)  # Why embedding size?
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask):
        attention = self.attention(query, key, value, mask) # How is this calling the forward method?

        x = self.dropout(self.norm1(attention + query))  # Adding Skip Connections
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

In [50]:
class Encoder(nn.Module):
    def __init__(  # Why isn't any input in the initialization? How is the input fed to the Encoder?
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length  # Maximum possible length of the sentence
    ):  # The above parameters are hyperparameters for the model that we are training
        super().__init__()
        self.embed_size = embed_size  # Do we really need another instance variable embed_size??
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)  # Is this good enough for big models?
        self.positional_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(embed_size, heads, dropout, forward_expansion) for _ in range(num_layers)
            ]
        )
        self.dropout = nn.Dropout(dropout)  # Why do we need this again? We don't seem to be using this in `forward`

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)  # What does .expand() do here?
        out = self.word_embedding(x) + self.positional_embedding(positions)  # Adding or concatenation?

        for layer in self.layers:
            out = layer(out, out, out, mask)  # What is the use of having separate matrices defined when we pass the same thing here?
            # Also why this? Can't we directly use the TransformerBlock instead of self.layers?
        
        return out


In [51]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super().__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, target_mask):  # Read about src and target masks
        attention = self.attention(x, x, x, target_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(query, key, value, src_mask)
        return out

In [52]:
class Decoder(nn.Module):
    def __init__(
        self,
        target_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length
    ):
        super().__init__()
        self.device = device
        self.word_embedding = nn.Embedding(target_vocab_size, embed_size)  # If we get the value embeddings from the previous layer, do we need this? (for seq2seq) - is my question sensible in the first place?
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device) for _ in range(num_layers)
            ]
        )

        self.fc_out = nn.Linear(embed_size, target_vocab_size)
        self.dropout = nn.Dropout(dropout)


    def forward(self, x, enc_out, src_mask, target_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout(self.word_embedding(x) + self.position_embedding(positions))  # Why dropout here?

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, target_mask)  # Why are we giving the same values here when we have separate matrices for keys and values?

        out = self.fc_out(x)
        return out

In [ ]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        target_vocab_size,
        src_pad_idx,  # Necessary to compute mask we are going to use
        target_pad_idx,
        embed_size = 256,
        num_layers = 6,
        forward_expansion = 4,
        heads = 8,
        dropout = 0,
        device = "cuda",
        max_length = 100
    ):
        super().__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length
        )

        self.decoder = Decoder(
            target_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length  # What is the purpose of this variable??
        )

        self.src_pad_idx = src_pad_idx
        self.target_pad_idx = target_pad_idx
        self.device =  device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)  # Look into the .unsqueeze() function
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_target_mask(self, target):
        N, target_len = target.shape
        target_mask = torch.tril(torch.ones((target_len, target_len))).expand(  # Triangular lower for masking
            N, 1, target_len, target_len  # To expand it to the training examples I guess?
        )
        return target_mask.to(self.device)  # Don't forget to return AND move to device!

    def forward(self, src, target):
        src_mask = self.make_src_mask(src)
        target_mask = self.make_target_mask(target)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(target, enc_src, src_mask, target_mask)
        return out

### Example

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(
    device
)
trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

src_pad_idx = 0
trg_pad_idx = 0
src_vocab_size = 10
trg_vocab_size = 10
model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(
    device
)
out = model(x, trg[:, :-1])
print(out)
print(out.shape)

cpu
tensor([[[-0.4076, -0.8250, -0.3485,  0.7102,  0.1573, -0.4691, -0.0984,
           0.3193,  0.6115, -0.0237],
         [ 0.0092,  0.0964,  0.8867, -0.3189,  0.2491, -1.0998, -0.0293,
           0.2697, -0.0599,  0.0719],
         [-0.9265, -0.1103,  0.8022, -0.3058,  0.1895, -0.5535, -0.1163,
           0.1099,  1.0126,  0.4507],
         [-0.1720,  0.1499, -0.0063, -0.1559, -0.1430, -0.9666,  0.3372,
           0.8199, -0.4038,  0.0190],
         [ 0.4433, -0.7454,  0.0722,  0.2000, -0.8239, -0.4571, -0.2999,
           0.6386, -0.1653,  0.5347],
         [ 0.9607, -0.2463,  0.6453,  0.2526, -0.2535, -0.6740,  0.0807,
           0.2469,  0.1038,  0.3005],
         [-0.5002, -0.8179,  0.3131,  0.4771, -0.4466, -0.6380, -0.1282,
           1.9126,  0.0919,  0.5859]],

        [[-0.3257, -0.8137, -0.2222,  0.7361,  0.3636, -0.4141, -0.3308,
           0.3201,  0.4566, -0.0326],
         [ 0.3360, -0.5969,  0.3104,  0.1160, -0.8658, -0.2600, -0.4460,
           0.6846,  0.3175, -0.32

# Training

## Loading the Dataset

We're using **SAMSum** — a dialogue summarization dataset with chat conversations and their summaries.

**What's in SAMSum?**
- `dialogue`: The chat conversation (what we feed to the **encoder**)
- `summary`: Short summary of the conversation (what we train the **decoder** to produce)
- `id`: Unique identifier

We'll take the first 1000 samples to keep training fast while learning.

In [5]:
from datasets import load_dataset

# Load SAMSum dataset (~2.5 MB download)
# We only load the 'train' split and take first 1000 samples
dataset = load_dataset("knkarthick/samsum", split="train[:1000]")

print(f"Dataset size: {len(dataset)} samples")
print(f"Features: {dataset.features}")

README.md: 0.00B [00:00, ?B/s]

c:\Users\abhij\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abhij\.cache\huggingface\hub\datasets--knkarthick--samsum. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14731 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Dataset size: 1000 samples
Features: {'id': Value('string'), 'dialogue': Value('string'), 'summary': Value('string')}


In [6]:
# Let's look at ONE example to understand what we're working with
sample = dataset[0]

print("=" * 60)
print("DIALOGUE (Input to Encoder):")
print("=" * 60)
print(sample['dialogue'])
print(f"\n[Length: {len(sample['dialogue'])} characters]")

print("\n" + "=" * 60)
print("SUMMARY (Target for Decoder):")
print("=" * 60)
print(sample['summary'])
print(f"\n[Length: {len(sample['summary'])} characters]")

DIALOGUE (Input to Encoder):
Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)

[Length: 92 characters]

SUMMARY (Target for Decoder):
Amanda baked cookies and will bring Jerry some tomorrow.

[Length: 56 characters]


In [7]:
# Let's understand the length distribution - this helps us set max_length later
dialogue_lengths = [len(d.split()) for d in dataset['dialogue']]  # Word count
summary_lengths = [len(s.split()) for s in dataset['summary']]

print("DIALOGUE lengths (in words):")
print(f"  Min: {min(dialogue_lengths)}, Max: {max(dialogue_lengths)}, Avg: {sum(dialogue_lengths)/len(dialogue_lengths):.0f}")

print("\nSUMMARY lengths (in words):")
print(f"  Min: {min(summary_lengths)}, Max: {max(summary_lengths)}, Avg: {sum(summary_lengths)/len(summary_lengths):.0f}")

DIALOGUE lengths (in words):
  Min: 9, Max: 447, Avg: 93

SUMMARY lengths (in words):
  Min: 3, Max: 58, Avg: 20
